
# CrewAI with AgentNeo Integration
This Jupyter notebook demonstrates the integration of AgentNeo, a powerful tracing and monitoring tool, with CrewAI, a framework for orchestrating role-playing AI agents. This integration allows for comprehensive analysis and debugging of AI-powered systems.

# Setup and Imports
First, let's import the necessary libraries and set up our environment.

In [ ]:
import os

In [ ]:
from crewai import Agent, Task, Crew, Process
from dotenv import load_dotenv

from agentneo import AgentNeo, Tracer ,Evaluation

# Load environment variables
load_dotenv("enter your .env path")


# Initialize AgentNeo Session and Tracer
Now, let's set up our AgentNeo session and tracer.

In [ ]:
# Initialize AgentNeo Session and Tracer
neo_session = AgentNeo(session_name="digital_marketing_campaign")

# Connect to a Project
neo_session.create_project(project_name="digital_marketing_project1")

# Create tracer
tracer = Tracer(session=neo_session)

tracer.start()


# Define Helper Functions
Let's define some helper functions for our AI tools, using AgentNeo's tracing capabilities.

In [ ]:
from langchain_community.tools import TavilySearchResults

seo_tool = tracer.wrap_langchain_tool(TavilySearchResults())

# Define Agents
Now, let's create our AI agents using CrewAI.

In [ ]:
from langchain.tools import Tool

# Define custom tools
content_tool = Tool(
    name="Content Creation",
    func=lambda input: f"Generating content based on input: {input}",
    description="Generates content for blogs or social media posts"
)

social_media_tool = Tool(
    name="Social Media Strategy",
    func=lambda input: f"Devising a social media strategy based on input: {input}",
    description="Provides a strategy for social media campaigns"
)

analytics_tool = Tool(
    name="Analytics",
    func=lambda input: f"Analyzing performance data for: {input}",
    description="Analyzes campaign performance"
)

# Wrap tools using tracer
content_tool = tracer.wrap_langchain_tool(content_tool)
social_media_tool = tracer.wrap_langchain_tool(social_media_tool)
analytics_tool = tracer.wrap_langchain_tool(analytics_tool)


In [ ]:
# Define Agents
@tracer.trace_agent("content_creator")
def create_content_creator():
    return Agent(
        role='Content Creator',
        goal='Produce engaging content for blogs and social media.',
        backstory="""You are a skilled content creator specializing in crafting compelling narratives.
        Your content drives user engagement and brand awareness.""",
        verbose=True,
        allow_delegation=False,
        tools=[content_tool],
    )

@tracer.trace_agent("social_media_manager")
def create_social_media_manager():
    return Agent(
        role='Social Media Manager',
        goal='Develop and execute a social media strategy to enhance brand presence.',
        backstory="""As a social media manager, you excel at building brand communities and engaging audiences.
        You utilize analytics to inform your strategies.""",
        verbose=True,
        allow_delegation=True,
        tools=[social_media_tool],
    )

@tracer.trace_agent("analytics_expert")
def create_analytics_expert():
    return Agent(
        role='Analytics Expert',
        goal='Analyze campaign performance and optimize strategies based on data.',
        backstory="""You are an analytics expert with a knack for interpreting data.
        You provide actionable insights that drive marketing decisions.""",
        verbose=True,
        allow_delegation=True,
        tools=[analytics_tool],
    )

content_creator = create_content_creator()
social_media_manager = create_social_media_manager()
analytics_expert = create_analytics_expert()



# Define Tasks
Let's create tasks for our agents.

In [ ]:
# Define Tasks
task1 = Task(
    description="""Create a series of blog posts that highlight the features and benefits of the new product.
    Ensure SEO optimization for key search terms.""",
    expected_output="A set of blog posts in markdown format",
    agent=content_creator
)

task2 = Task(
    description="""Develop a comprehensive social media strategy for the product launch,
    including content calendar and engagement tactics.""",
    expected_output="Social media strategy document",
    agent=social_media_manager
)

task3 = Task(
    description="""Analyze the initial performance metrics of the marketing campaign,
    providing insights and recommendations for adjustments.""",
    expected_output="Analysis report with recommendations",
    agent=analytics_expert
)


# Create and Execute Crew
Now, let's create our crew and execute the tasks.

In [ ]:
# Create and Execute Crew
crew = Crew(
    agents=[content_creator, social_media_manager, analytics_expert],
    tasks=[task1, task2, task3],
    process=Process.sequential,
    verbose=True,
)
result = crew.kickoff()

print(result)
tracer.stop()

# Metrics Evaluation

In [ ]:
exe = Evaluation(session=neo_session, trace_id=tracer.trace_id)

# run a single metric
exe.evaluate(metric_list=['goal_decomposition_efficiency', 
                         'goal_fulfillment_rate', 
                         'tool_call_correctness_rate', 
                         'tool_call_success_rate'])

In [ ]:
results = exe.get_results()
results

## Launch Dashboard

In [ ]:
neo_session.launch_dashboard(port=3000)